In [1]:
import requests
import sys
import json
import base64
import time
import logging
from solders.keypair import Keypair
from solders.transaction import VersionedTransaction 
from solana.rpc.api import Client
from solana.rpc.types import TxOpts
import dontshare as d 
from pprint import pprint
from functools import lru_cache
from datetime import datetime, timedelta
import pytz
import json
import pandas as pd
from pprint import pprint 
from IPython.display import display, HTML
import time

In [2]:
API_Key = d.birdeye
wallet = d.sol_wallet
chain = "solana"
multichain = "solana,ethereum,bsc"
address = "EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm" #WIF

### DEFI APIs

OHLCV Data

In [4]:
def get_ohlcv_data_multi(tokens, API_Key, timeframes=None):
    """
    Fetch OHLCV data for multiple tokens and specified timeframes.

    Args:
    tokens (list): List of token addresses to fetch data for.
    API_Key (str): Your Birdeye API key.
    timeframes (list): List of timeframes to fetch data for. Default is ['15m'].

    Returns:
    dict: A nested dictionary with tokens and timeframes as keys and DataFrames as values.
    """
    if timeframes is None:
        timeframes = ['15m']

    timeframes_data = {
        '1m': 1 * 24 * 60,    # 1 day of 1-minute data
        '3m': 3 * 24 * 20,    # 3 days of 3-minute data
        '5m': 7 * 24 * 12,    # 7 days of 5-minute data
        '15m': 25 * 24 * 4,   # 25 days of 15-minute data
        '30m': 50 * 24 * 2,   # 50 days of 30-minute data
        '1H': 100 * 24,       # 100 days of 1-hour data
        '2H': 200 * 12,       # 200 days of 2-hour data
        '4H': 400 * 6,        # 400 days of 4-hour data
        '6H': 600 * 4,        # 600 days of 6-hour data
        '8H': 800 * 3,        # 800 days of 8-hour data
        '12H': 1200 * 2,      # 1200 days of 12-hour data
        '1D': 5 * 365         # 5 years of daily data
    }

    results = {}

    for token in tokens:
        results[token] = {}
        for timeframe in timeframes:
            if timeframe not in timeframes_data:
                print(f"Invalid timeframe: {timeframe}. Skipping.")
                continue

            # Calculate start and end times
            end_time = datetime.now(pytz.UTC)
            start_time = end_time - timedelta(hours=timeframes_data[timeframe])

            # Convert to Unix timestamps
            time_from = int(start_time.timestamp())
            time_to = int(end_time.timestamp())

            url = f"https://public-api.birdeye.so/defi/ohlcv?address={token}&type={timeframe}&time_from={time_from}&time_to={time_to}"

            headers = {
                "accept": "application/json",
                "X-API-KEY": API_Key
            }

            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                try:
                    data = json.loads(response.text)
                    
                    if 'data' in data and 'items' in data['data']:
                        items = data['data']['items']
                        
                        if items:
                            df = pd.DataFrame(items)
                            df['datetime'] = pd.to_datetime(df['unixTime'], unit='s')
                            df.set_index('datetime', inplace=True)
                            columns_order = [col for col in df.columns if col != 'unixTime']
                            df = df[columns_order]

                            results[token][timeframe] = df
                            
                            # Print the head of each token-timeframe table
                            print(f"\nHead of {token} - {timeframe} table:")
                            print(df.head())
                        else:
                            results[token][timeframe] = pd.DataFrame()
                            print(f"\nNo data items found for token {token}, timeframe {timeframe}.")
                    else:
                        results[token][timeframe] = pd.DataFrame()
                        print(f"\nUnexpected response structure for token {token}, timeframe {timeframe}.")
                except json.JSONDecodeError:
                    results[token][timeframe] = pd.DataFrame()
                    print(f"\nFailed to parse JSON response for token {token}, timeframe {timeframe}.")
            else:
                results[token][timeframe] = pd.DataFrame()
                print(f"\nRequest failed with status code: {response.status_code} for token {token}, timeframe {timeframe}.")

    return results

# Example usage:
tokens = ['EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm', 'So11111111111111111111111111111111111111112']
result = get_ohlcv_data_multi(tokens, API_Key, timeframes=['1m', '3m']) # ['1m', '3m', '5m', '15m', '30m', '1H', '2H', '4H', '6H', '8H', '12H', '1D']


Head of EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm - 1m table:
                                                          address         c  \
datetime                                                                      
2024-08-21 17:15:00  EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm  1.501753   
2024-08-21 17:16:00  EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm  1.502773   
2024-08-21 17:17:00  EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm  1.505043   
2024-08-21 17:18:00  EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm  1.505184   
2024-08-21 17:19:00  EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm  1.509527   

                            h         l         o type             v  
datetime                                                              
2024-08-21 17:15:00  1.504895  1.501753  1.504523   1m   2309.182075  
2024-08-21 17:16:00  1.502773  1.501070  1.501753   1m   4816.822118  
2024-08-21 17:17:00  1.505043  1.502773  1.502773   1m  16967.572898  
2024-08-21 17:18:00  1.50

### Token APIs

Market Data

In [5]:
def get_token_market_data(address, API_Key):
    """
    Fetch market data for a specific token and return it as a pandas DataFrame.

    Args:
    address (str): The token address to fetch market data for.
    API_Key (str): Your Birdeye API key.

    Returns:
    pd.DataFrame: A DataFrame containing the token's market data.
    """
    url = f"https://public-api.birdeye.so/defi/v3/token/market-data?address={address}"

    headers = {
        "accept": "application/json",
        "X-API-KEY": API_Key
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if 'data' in data and data['success']:
            # Convert the data to a DataFrame
            df = pd.DataFrame([data['data']])
            
            # Rename columns for clarity
            df = df.rename(columns={
                'address': 'Address',
                'price': 'Price',
                'liquidity': 'Liquidity',
                'supply': 'Total Supply',
                'marketcap': 'Market Cap',
                'circulating_supply': 'Circulating Supply',
                'circulating_marketcap': 'Circulating Market Cap'
            })
            
            return df
        else:
            print("Unexpected response structure or request unsuccessful")
            return pd.DataFrame()
    else:
        print(f"Request failed with status code: {response.status_code}")
        return pd.DataFrame()

# Example usage:
address = "So11111111111111111111111111111111111111112"
market_data_df = get_token_market_data(address, API_Key)
market_data_df

,Address,Price,Liquidity,Total Supply,Market Cap,Circulating Supply,Circulating Market Cap
0,So11111111111111111111111111111111111111112,160.388756,1.521629e+10,5.868466e+08,9.412359e+10,5.868466e+08,9.412359e+10


Token List

In [5]:
# Example usage:
sort_by = "mc"  # mc, v24hUSD, v24hChangePercent 
sort_type = "desc"  # asc, desc
min_liquidity = 100000  # Minimum liquidity in USD (100k)
min_volume_24h = 1000000  # Minimum 24-hour trading volume in USD
min_market_cap = 1000000  # Minimum market cap in USD (1 million)
max_market_cap = 900000000  # Maximum market cap in USD (900 million)
total_tokens = 1000  # Change this to the number of tokens you want to retrieve
chain = "solana"  # Choose between: solana,ethereum,arbitrum,avalanche,bsc,optimism,polygon,base,zksync

def get_token_list(sort_by, sort_type, min_liquidity, min_volume_24h, min_market_cap, max_market_cap, total_tokens, chain, API_Key):
    """
    Fetch and filter token list based on specified criteria.

    Args:
        sort_by (str): Criterion to sort by (e.g., 'mc', 'rank', 'v24hUSD', 'v24hChangePercent').
        sort_type (str): Sort order ('asc' or 'desc').
        min_liquidity (float): Minimum liquidity in USD.
        min_volume_24h (float): Minimum 24-hour trading volume in USD.
        min_market_cap (float): Minimum market cap in USD.
        max_market_cap (float): Maximum market cap in USD.
        total_tokens (int): Number of tokens to retrieve.
        chain (str): Blockchain to query.
        API_Key (str): Your Birdeye API key.

    Returns:
        pd.DataFrame: A DataFrame containing the filtered token list.
    """
    limit = 50
    all_tokens = []

    for offset in range(0, total_tokens * 3, limit):  # Fetch more tokens to account for filtering
        url = f"https://public-api.birdeye.so/defi/tokenlist?sort_by={sort_by}&sort_type={sort_type}&offset={offset}&limit={limit}&min_liquidity={min_liquidity}"

        headers = {
            "accept": "application/json",
            "x-chain": chain,
            "X-API-KEY": API_Key
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = json.loads(response.text)
            if 'data' in data and 'tokens' in data['data']:
                new_tokens = [
                    {
                        'name': token.get('name'),
                        'symbol': token.get('symbol'),
                        'mc': token.get('mc'),
                        'v24hUSD': token.get('v24hUSD'),
                        'v24hChangePercent': token.get('v24hChangePercent'),
                        'liquidity': token.get('liquidity'),
                        'address': token.get('address')
                    }
                    for token in data['data']['tokens']
                    if token.get('v24hUSD', 0) >= min_volume_24h
                    and min_market_cap <= token.get('mc', 0) <= max_market_cap
                ]
                all_tokens.extend(new_tokens)
            else:
                print(f"Unexpected response structure at offset {offset}")
                break
        else:
            print(f"Request failed with status code: {response.status_code} at offset {offset}")
            break

        if len(all_tokens) >= total_tokens:
            break

        # Respect rate limits
        time.sleep(1)

    all_tokens = all_tokens[:total_tokens]

    if all_tokens:
        df = pd.DataFrame(all_tokens)

        # Select and rename columns
        columns_to_display = {
            'name': 'Name',
            'symbol': 'Symbol',
            'mc': 'Market Cap',
            'v24hUSD': 'Volume 24h',
            'v24hChangePercent': '24h Change (%)',
            'liquidity': 'Liquidity',
            'address': 'Address'
        }
        
        # Only include columns that exist in the DataFrame
        available_columns = [col for col in ['name', 'symbol', 'mc', 'v24hUSD', 'v24hChangePercent', 'liquidity', 'address'] if col in df.columns]
        df = df[available_columns].rename(columns={col: columns_to_display[col] for col in available_columns})

        return df
    else:
        return pd.DataFrame()

df = get_token_list(sort_by, sort_type, min_liquidity, min_volume_24h, min_market_cap, max_market_cap, total_tokens, chain, API_Key)
display(df)

,Name,Symbol,Market Cap,Volume 24h,24h Change (%),Liquidity,Address
0,Baby Goatseus,BABYGOAT,8.656751e+08,1.010300e+06,NaN,2.603512e+05,E75qcuWFZ8eHxXhgY55VEQC7dBdkqwjVfkQizRBTUyh8
1,Marinade staked SOL (mSOL),mSOL,8.254507e+08,1.169932e+07,-49.850888,6.077107e+07,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So
2,cat in a dogs world,MEW,7.602484e+08,4.140591e+06,-75.770743,5.114039e+07,MEW1gQWJ3nEXg2qgERiKu7FAFj79PHvQVREQUzScPP5
3,Jupiter Perps LP,JLP,7.546884e+08,1.831428e+07,-48.116217,1.519830e+07,27G8MtK7VtTcCHkpASjSDdkWWYfoqT6ggEuKidVJidD4
4,Kima,KIMA,6.734893e+08,9.230451e+07,-54.213218,2.625478e+08,BsQCC4D2AZhC9RctuugBKLCWaNycwmZTzwpUjgGHXWbw
...,...,...,...,...,...,...,...
129,Catana,CATANA,1.686873e+06,1.772867e+06,29.941868,1.824735e+05,GmbC2HgWpHpq9SHnmEXZNT5e1zgcU9oASDqbAkGTpump
130,Hachiko,HACHI,1.461687e+06,3.685842e+06,-71.091736,2.898018e+05,4amstKcbziHCqwev9esMtRGDTdjHSviiNXT7WtajgjUq
131,Elysium,Elysium,1.077103e+06,1.778767e+07,NaN,1.515028e+05,9Za5hA1XFyGBNbGNEJH7v411AXaW19WMhKaAvamUgT7T
132,just a normal dog,maxy,1.076068e+06,1.368244e+07,NaN,1.415662e+05,2tBPEZp3uChtKvdKhWgaA8AsqK3J6Mvt8w7XQo39pump


Token Security

In [5]:
def get_token_security_data_multi(tokens, API_Key):
    """
    Fetch token security data for multiple tokens.

    Args:
    tokens (list): List of token addresses to fetch data for.
    API_Key (str): Your Birdeye API key.

    Returns:
    dict: A dictionary with token addresses as keys and their security data as values.
    """
    results = {}

    for token in tokens:
        url = f"https://public-api.birdeye.so/defi/token_security?address={token}"

        headers = {
            "accept": "application/json",
            "X-API-KEY": API_Key
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = json.loads(response.text)
            if 'data' in data:
                # Extract the specific values
                token_data = data['data']
                selected_data = {
                    'top10HolderPercent': token_data.get('top10HolderPercent', 'N/A'),
                    'top10UserPercent': token_data.get('top10UserPercent', 'N/A'),
                    'preMarketHolder': token_data.get('preMarketHolder', 'N/A'),
                    'mutableMetadata': token_data.get('mutableMetadata', 'N/A'),
                    'creatorPercentage': token_data.get('creatorPercentage', 'N/A'),
                    'ownerPercentage': token_data.get('ownerPercentage', 'N/A')
                }

                # Create a DataFrame from the selected data
                df_selected = pd.DataFrame.from_dict(selected_data, orient='index', columns=['Value'])
                df_selected.index.name = 'Attribute'
                df_selected.reset_index(inplace=True)

                # Create full data DataFrame
                df_full = pd.json_normalize(data['data'])
                df_full = df_full.map(lambda x: json.dumps(x, indent=2) if isinstance(x, (dict, list)) else x)
                df_full_pivoted = df_full.T.reset_index()
                df_full_pivoted.columns = ['Attribute', 'Value']

                # Store both selected and full data for the token
                results[token] = {
                    'selected_data': df_selected,
                    'full_data': df_full_pivoted
                }
            else:
                results[token] = {
                    'error': "Unexpected response structure. 'data' key not found.",
                    'data': json.dumps(data, indent=2)
                }
        else:
            results[token] = {
                'error': f"Request failed with status code: {response.status_code}",
                'data': response.text
            }

        # Respect rate limits by adding a small delay between requests
        time.sleep(1)

    return results

# Example usage:
tokens = ['EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm', 'So11111111111111111111111111111111111111112']
result = get_token_security_data_multi(tokens, API_Key)

for token, data in result.items():
    print(f"\nToken: {token}")
    if 'selected_data' in data:
        print("Selected Data:")
        display(data['selected_data'])
    else:
        print(f"Error: {data['error']}")


Token: EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm
Selected Data:


,Attribute,Value
0,top10HolderPercent,0.392301
1,top10UserPercent,0.392301
2,preMarketHolder,[]
3,mutableMetadata,False
4,creatorPercentage,0.0
5,ownerPercentage,None



Token: So11111111111111111111111111111111111111112
Selected Data:


,Attribute,Value
0,top10HolderPercent,0.008482
1,top10UserPercent,0.007844
2,preMarketHolder,[]
3,mutableMetadata,True
4,creatorPercentage,None
5,ownerPercentage,None


Token Overview

In [6]:
def get_token_overview_data_multi(tokens, API_Key):
    """
    Fetch token overview data for multiple tokens.

    Args:
    tokens (list): List of token addresses to fetch data for.
    API_Key (str): Your Birdeye API key.

    Returns:
    dict: A dictionary with token addresses as keys and their overview data as pandas DataFrames.
    """
    results = {}

    for address in tokens:
        url = f"https://public-api.birdeye.so/defi/token_overview?address={address}"

        headers = {
            "accept": "application/json",
            "X-API-KEY": API_Key
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = json.loads(response.text)
            if 'data' in data:
                token_data = data['data']
                selected_data = {
                    'address': token_data.get('address', 'N/A'),
                    'decimals': token_data.get('decimals', 'N/A'),
                    'symbol': token_data.get('symbol', 'N/A'),
                    'name': token_data.get('name', 'N/A'),
                    'price': token_data.get('price', 'N/A'),
                    'liquidity': token_data.get('liquidity', 'N/A'),
                    'uniqueWallet30m': token_data.get('uniqueWallet30m', 'N/A'),
                    'uniqueWallet30mChangePercent': token_data.get('uniqueWallet30mChangePercent', 'N/A'),
                    'uniqueWallet1h': token_data.get('uniqueWallet1h', 'N/A'),
                    'uniqueWallet1hChangePercent': token_data.get('uniqueWallet1hChangePercent', 'N/A'),
                    'uniqueWallet2h': token_data.get('uniqueWallet2h', 'N/A'),
                    'uniqueWallet2hChangePercent': token_data.get('uniqueWallet2hChangePercent', 'N/A'),
                    'uniqueWallet4h': token_data.get('uniqueWallet4h', 'N/A'),
                    'uniqueWallet4hChangePercent': token_data.get('uniqueWallet4hChangePercent', 'N/A'),
                    'uniqueWallet6h': token_data.get('uniqueWallet6h', 'N/A'),
                    'uniqueWallet6hChangePercent': token_data.get('uniqueWallet6hChangePercent', 'N/A'),
                    'uniqueWallet8h': token_data.get('uniqueWallet8h', 'N/A'),
                    'uniqueWallet8hChangePercent': token_data.get('uniqueWallet8hChangePercent', 'N/A'),
                    'uniqueWallet12h': token_data.get('uniqueWallet12h', 'N/A'),
                    'uniqueWallet12hChangePercent': token_data.get('uniqueWallet12hChangePercent', 'N/A'),
                    'uniqueWallet24h': token_data.get('uniqueWallet24h', 'N/A'),
                    'uniqueWallet24hChangePercent': token_data.get('uniqueWallet24hChangePercent', 'N/A'),
                    'realMc': token_data.get('realMc', 'N/A'),
                    'holder': token_data.get('holder', 'N/A'),
                    'numberMarkets': token_data.get('numberMarkets', 'N/A'),
                    'coingeckoId': token_data.get('extensions', {}).get('coingeckoId', 'N/A'),
                    'telegram': token_data.get('extensions', {}).get('telegram', 'N/A'),
                    'twitter': token_data.get('extensions', {}).get('twitter', 'N/A'),
                    'website': token_data.get('extensions', {}).get('website', 'N/A'),
                    'discord': token_data.get('extensions', {}).get('discord', 'N/A')
                }

                # Create a DataFrame from the selected data
                df = pd.DataFrame.from_dict(selected_data, orient='index', columns=['Value'])
                df.index.name = 'Attribute'
                df.reset_index(inplace=True)

                results[address] = df
            else:
                results[address] = pd.DataFrame({'Error': ["Unexpected response structure. 'data' key not found."]})
        else:
            results[address] = pd.DataFrame({'Error': [f"Request failed with status code: {response.status_code}"]})

        # Respect rate limits by adding a small delay between requests
        time.sleep(1)

    return results

# Example usage:
tokens = ['EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm', 'So11111111111111111111111111111111111111112']
result = get_token_overview_data_multi(tokens, API_Key)

for token, df in result.items():
    print(f"\nToken: {token}")
    display(df)


Token: EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm


,Attribute,Value
0,address,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm
1,decimals,6
2,symbol,$WIF
3,name,dogwifhat
4,price,2.611047
5,liquidity,37228316.533126
6,uniqueWallet30m,464
7,uniqueWallet30mChangePercent,10.739857
8,uniqueWallet1h,734
9,uniqueWallet1hChangePercent,21.322314



Token: So11111111111111111111111111111111111111112


,Attribute,Value
0,address,So11111111111111111111111111111111111111112
1,decimals,9
2,symbol,SOL
3,name,Wrapped SOL
4,price,156.370568
5,liquidity,15426985071.787617
6,uniqueWallet30m,87066
7,uniqueWallet30mChangePercent,-0.926263
8,uniqueWallet1h,149587
9,uniqueWallet1hChangePercent,6.452462


Token Trade Data (Single)

In [7]:
def get_token_trade_data_multi(tokens, API_Key):
    """
    Fetch token trade data for multiple tokens.

    Args:
    tokens (list): List of token addresses to fetch data for.
    API_Key (str): Your Birdeye API key.

    Returns:
    dict: A dictionary with token addresses as keys and their trade data as pandas DataFrames.
    """
    results = {}

    for address in tokens:
        url = f"https://public-api.birdeye.so/defi/v3/token/trade-data/single?address={address}"

        headers = {
            "accept": "application/json",
            "X-API-KEY": API_Key
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = json.loads(response.text)
            if 'data' in data:
                token_data = data['data']
                
                # Convert the token data to a DataFrame
                df = pd.DataFrame([token_data])
                
                # Transpose the DataFrame for better readability
                df_transposed = df.T.reset_index()
                df_transposed.columns = ['Attribute', 'Value']
                
                results[address] = df_transposed
            else:
                results[address] = pd.DataFrame({'Attribute': ['Error'], 'Value': ["Unexpected response structure. 'data' key not found."]})
        else:
            results[address] = pd.DataFrame({'Attribute': ['Error'], 'Value': [f"Request failed with status code: {response.status_code}"]})

        # Respect rate limits by adding a small delay between requests
        time.sleep(1)

    return results

# Example usage:
tokens = ['So11111111111111111111111111111111111111112', 'EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm']
result = get_token_trade_data_multi(tokens, API_Key)

for token, df in result.items():
    print(f"\nToken Trade Data for {token}:")
    display(df)


Token Trade Data for So11111111111111111111111111111111111111112:


,Attribute,Value
0,address,So11111111111111111111111111111111111111112
1,holder,710597
2,market,56670
3,last_trade_unix_time,1729356435
4,last_trade_human_time,2024-10-19T16:47:15
...,...,...
179,volume_sell_24h,7973479.010225
180,volume_sell_24h_usd,1238183249.55175
181,volume_sell_history_24h,8548778.035505
182,volume_sell_history_24h_usd,1275697813.732954



Token Trade Data for EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm:


,Attribute,Value
0,address,EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm
1,holder,189826
2,market,216
3,last_trade_unix_time,1729356435
4,last_trade_human_time,2024-10-19T16:47:15
...,...,...
179,volume_sell_24h,4630244.419019
180,volume_sell_24h_usd,19602851.614899
181,volume_sell_history_24h,10203551.961089
182,volume_sell_history_24h_usd,32079071.374731


Trending List

In [8]:
def get_trending_tokens(total_tokens, API_Key, chain):
    """
    Fetch trending tokens data.

    Args:
    total_tokens (int): Number of trending tokens to fetch.
    API_Key (str): Your Birdeye API key.
    chain (str): The blockchain to fetch data for (e.g., 'solana').

    Returns:
    pd.DataFrame: A DataFrame containing the trending tokens data.
    """
    sort_by = "volume24hUSD"  # rank, volume24hUSD, liquidity
    sort_type = "desc"  # asc, desc
    limit = 20  # API limit per request
    all_tokens = []

    for offset in range(0, total_tokens, limit):
        url = f"https://public-api.birdeye.so/defi/token_trending?sort_by={sort_by}&sort_type={sort_type}&offset={offset}&limit={limit}"

        headers = {
            "accept": "application/json",
            "x-chain": chain,
            "X-API-KEY": API_Key
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = json.loads(response.text)
            if 'data' in data and 'tokens' in data['data']:
                all_tokens.extend(data['data']['tokens'])
            else:
                print(f"Unexpected response structure at offset {offset}")
                print("Response:", response.text)  # Print the response for debugging
        else:
            print(f"Request failed with status code: {response.status_code} at offset {offset}")
            print("Response:", response.text)  # Print the response for debugging
            break

        if len(all_tokens) >= total_tokens:
            break

        # Respect rate limits
        time.sleep(1)

    all_tokens = all_tokens[:total_tokens]

    if all_tokens:
        df = pd.DataFrame(all_tokens)

        # Select and rename columns
        columns_to_display = {
            'symbol': 'Symbol',
            'name': 'Name',
            'price': 'Price',
            'volume24hUSD': 'Volume 24h',
            'liquidity': 'Liquidity',
            'rank': 'Rank'
        }
        df = df[columns_to_display.keys()].rename(columns=columns_to_display)

        # Ensure we have exactly the requested number of rows
        df = df.head(total_tokens)

        return df
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no data is available

# Example usage:
number_of_trending_tokens = 100
trending_tokens_df = get_trending_tokens(number_of_trending_tokens, API_Key, chain)
print(trending_tokens_df)

      Symbol              Name      Price    Volume 24h     Liquidity  Rank
0      GRA$$             GRA$$  68.828500  3.445626e+08  2.561562e+08   477
1   AutoLaye          AutoLaye   1.581385  3.325718e+08  1.250596e+09   498
2     $croll            $croll   4.072012  1.696432e+08  2.954270e+08   549
3       GNON         numogram    0.105898  1.587160e+08  2.676557e+06    53
4       GOAT  Goatseus Maximus   0.477926  1.404811e+08  1.430131e+07    51
..       ...               ...        ...           ...           ...   ...
95     GMika             GMika   0.001124  2.514325e+06  2.916738e+05   854
96      GROK           GROK AI   0.000011  2.482968e+06  1.230521e+04   117
97  numogram     ccru numogram   0.000017  2.404393e+06  1.484691e+04   116
98    KEYDOG            keydog   0.004772  2.267543e+06  3.555868e+05    26
99     GOVAI       Pigeon Tech   0.001083  2.237407e+06  1.610820e+05   821

[100 rows x 6 columns]


New Listings 0-3 days max

In [3]:
from datetime import datetime, timedelta
import requests
import pandas as pd
import pytz
import time

def get_new_listings(days_back, hours_back, minutes_back, API_Key, liquidity_filter):
    """
    Fetch new token listings data with optimized performance and enhanced verification.
    """
    # Calculate end and start times (all in UTC)
    end_time = datetime.now(pytz.UTC)
    start_time = end_time - timedelta(days=days_back, hours=hours_back, minutes=minutes_back)
    
    # Convert to Unix timestamp
    current_timestamp = int(end_time.timestamp())
    start_timestamp = int(start_time.timestamp())
    
    # Add overlap protection
    start_timestamp -= 60  # Add 1 minute overlap
    last_token_time = current_timestamp + 60  # Add 1 minute overlap
    
    all_tokens = []
    page_size = 10  # API limit is 10
    consecutive_old_tokens = 0  # Counter for old tokens
    total_requests = 0
    
    print(f"\nFetching tokens from {start_time} to {end_time}")
    print(f"Start timestamp: {start_timestamp}")
    print(f"End timestamp: {current_timestamp}")
    
    with requests.Session() as session:
        session.headers.update({
            "accept": "application/json",
            "X-API-KEY": API_Key,
            "x-chain": "solana"
        })
        
        while True:
            try:
                url = f"https://public-api.birdeye.so/defi/v2/tokens/new_listing?time_to={last_token_time}&limit={page_size}&meme_platform_enabled=true"
                response = session.get(url)
                response.raise_for_status()
                total_requests += 1
                
                data = response.json()
                if 'data' not in data or 'items' not in data['data']:
                    print("\nNo more data available from API")
                    break
                    
                new_tokens = data['data']['items']
                if not new_tokens:
                    print("\nNo more tokens available")
                    break
                
                tokens_in_window = 0
                for token in new_tokens:
                    token_time = int(datetime.fromisoformat(token['liquidityAddedAt'].replace('Z', '+00:00')).timestamp())
                    if start_timestamp <= token_time <= current_timestamp:
                        all_tokens.append(token)
                        tokens_in_window += 1
                    last_token_time = min(last_token_time, token_time)
                
                if tokens_in_window == 0:
                    consecutive_old_tokens += 1
                    if consecutive_old_tokens >= 30:  # Increased from 20 to 30
                        print(f"\nNo new tokens found in last 30 batches, stopping search")
                        break
                else:
                    consecutive_old_tokens = 0
                
                print(f"Fetched {len(all_tokens)} tokens (Current batch time: {datetime.fromtimestamp(last_token_time, tz=pytz.UTC)})", end='\r')
                
                time.sleep(0.08)  # Rate limiting
                
            except Exception as e:
                print(f"\nError: {e}")
                break
    
    print(f"\n\nTotal API requests made: {total_requests}")
    print(f"Total tokens fetched: {len(all_tokens)}")
    
    if not all_tokens:
        return pd.DataFrame(), pd.DataFrame()
    
    # Convert to DataFrame
    df_full = pd.DataFrame(all_tokens)
    
    # Convert timestamp to datetime
    df_full['liquidityAddedAt'] = pd.to_datetime(df_full['liquidityAddedAt'])
    
    print("\nTime range of fetched tokens:")
    print(f"Earliest: {df_full['liquidityAddedAt'].min()}")
    print(f"Latest: {df_full['liquidityAddedAt'].max()}")
    
    # Apply filters
    mask = (
        df_full['name'].notna() & 
        (df_full['name'] != 'None') & 
        df_full['logoURI'].notna() & 
        (df_full['logoURI'] != 'None') & 
        (df_full['liquidity'].astype(float) >= liquidity_filter)
    )
    df_filtered = df_full[mask]
    
    return df_full, df_filtered

# Example usage:
days_back = 0  # Increased to 2 days
hours_back = 12
minutes_back = 0
liquidity_filter = 10

print("Fetching token data...")
df_full, df_filtered = get_new_listings(days_back, hours_back, minutes_back, API_Key, liquidity_filter)

# Save to CSV only if we have data
if not df_full.empty:
    df_full.to_csv('granular_new_listings.csv', index=False)
    print("\nData saved to granular_new_listings.csv")

Fetching token data...

Fetching tokens from 2024-12-05 15:34:18.627009+00:00 to 2024-12-05 16:34:18.627009+00:00
Start timestamp: 1733412798
End timestamp: 1733416458


Fetched 2709 tokens (Current batch time: 2024-12-05 15:27:21+00:00)
No new tokens found in last 30 batches, stopping search


Total API requests made: 301
Total tokens fetched: 2709

Time range of fetched tokens:
Earliest: 2024-12-05 15:33:18
Latest: 2024-12-05 16:34:11

Data saved to granular_new_listings.csv


In [7]:
from datetime import datetime, timedelta
import requests
import pandas as pd
import pytz
import time

def get_new_listings(days_back, hours_back, minutes_back, API_Key, liquidity_filter):
    """
    Fetch new token listings data with optimized performance and enhanced verification.
    """
    # Calculate end and start times (all in UTC)
    end_time = datetime.now(pytz.UTC)
    start_time = end_time - timedelta(days=days_back, hours=hours_back, minutes=minutes_back)
    
    # Convert to Unix timestamp
    current_timestamp = int(end_time.timestamp())
    start_timestamp = int(start_time.timestamp())
    
    # Add overlap protection
    start_timestamp -= 60  # Add 1 minute overlap
    last_token_time = current_timestamp + 60  # Add 1 minute overlap
    
    all_tokens = []
    page_size = 10  # API limit is 10
    consecutive_old_tokens = 0  # Counter for old tokens
    total_requests = 0
    
    print(f"\nFetching tokens from {start_time} to {end_time}")
    print(f"Start timestamp: {start_timestamp}")
    print(f"End timestamp: {current_timestamp}")
    
    with requests.Session() as session:
        session.headers.update({
            "accept": "application/json",
            "X-API-KEY": API_Key,
            "x-chain": "solana"
        })
        
        while True:
            try:
                url = f"https://public-api.birdeye.so/defi/v2/tokens/new_listing?time_to={last_token_time}&limit={page_size}&meme_platform_enabled=true"
                response = session.get(url)
                response.raise_for_status()
                total_requests += 1
                
                data = response.json()
                if 'data' not in data or 'items' not in data['data']:
                    print("\nNo more data available from API")
                    break
                    
                new_tokens = data['data']['items']
                if not new_tokens:
                    print("\nNo more tokens available")
                    break
                
                tokens_in_window = 0
                for token in new_tokens:
                    token_time = int(datetime.fromisoformat(token['liquidityAddedAt'].replace('Z', '+00:00')).timestamp())
                    if start_timestamp <= token_time <= current_timestamp:
                        all_tokens.append(token)
                        tokens_in_window += 1
                    last_token_time = min(last_token_time, token_time)
                
                if tokens_in_window == 0:
                    consecutive_old_tokens += 1
                    if consecutive_old_tokens >= 30:  # Increased from 20 to 30
                        print(f"\nNo new tokens found in last 30 batches, stopping search")
                        break
                else:
                    consecutive_old_tokens = 0
                
                print(f"Fetched {len(all_tokens)} tokens (Current batch time: {datetime.fromtimestamp(last_token_time, tz=pytz.UTC)})", end='\r')
                
                time.sleep(0.08)  # Rate limiting
                
            except Exception as e:
                print(f"\nError: {e}")
                break
    
    print(f"\n\nTotal API requests made: {total_requests}")
    print(f"Total tokens fetched: {len(all_tokens)}")
    
    if not all_tokens:
        return pd.DataFrame(), pd.DataFrame()
    
    # Convert to DataFrame
    df_full = pd.DataFrame(all_tokens)
    
    # Convert timestamp to datetime
    df_full['liquidityAddedAt'] = pd.to_datetime(df_full['liquidityAddedAt'])
    
    print("\nTime range of fetched tokens:")
    print(f"Earliest: {df_full['liquidityAddedAt'].min()}")
    print(f"Latest: {df_full['liquidityAddedAt'].max()}")
    
    # Apply filters
    mask = (
        df_full['name'].notna() & 
        (df_full['name'] != 'None') & 
        df_full['logoURI'].notna() & 
        (df_full['logoURI'] != 'None') & 
        (df_full['liquidity'].astype(float) >= liquidity_filter)
    )
    df_filtered = df_full[mask]
    
    return df_full, df_filtered

# Example usage:
days_back = 0  # Increased to 2 days
hours_back = 1
minutes_back = 0
liquidity_filter = 10

print("Fetching token data...")
df_full, df_filtered = get_new_listings(days_back, hours_back, minutes_back, API_Key, liquidity_filter)

# Save to CSV only if we have data
if not df_full.empty:
    df_full.to_csv('granular_new_listings.csv', index=False)
    print("\nData saved to granular_new_listings.csv")

Fetching token data...

Fetching tokens from 2024-12-05 11:51:44.057859+00:00 to 2024-12-05 12:51:44.057859+00:00
Start timestamp: 1733399384
End timestamp: 1733403104

First pass - collecting tokens backwards from end time...


Fetched 1489 tokens (Current batch time: 2024-12-05 11:36:08+00:00)
No new tokens found in last 30 batches, stopping search

Second pass - verification from start time...


In [3]:
from datetime import datetime, timedelta
import requests
import pandas as pd
import pytz
import time

def get_new_listings_around_time(target_unix_time, time_window_minutes, API_Key, liquidity_filter):
    """
    Fetch new token listings data around a specific time with optimized performance and enhanced verification.
    """
    # Calculate the start and end times for the search window
    target_time = datetime.fromtimestamp(target_unix_time, pytz.UTC)
    start_time = target_time - timedelta(minutes=time_window_minutes)
    end_time = target_time + timedelta(minutes=time_window_minutes)
    
    # Convert to Unix timestamp
    start_timestamp = int(start_time.timestamp())
    end_timestamp = int(end_time.timestamp())
    
    all_tokens = []
    page_size = 10  # API limit is 10
    last_token_time = end_timestamp
    total_requests = 0
    
    print(f"\nFetching tokens around {target_time} with a window of {time_window_minutes} minutes")
    print(f"Start timestamp: {start_timestamp}")
    print(f"End timestamp: {end_timestamp}")
    
    with requests.Session() as session:
        session.headers.update({
            "accept": "application/json",
            "X-API-KEY": API_Key,
            "x-chain": "solana"
        })
        
        while True:
            try:
                url = f"https://public-api.birdeye.so/defi/v2/tokens/new_listing?time_to={last_token_time}&limit={page_size}&meme_platform_enabled=true"
                response = session.get(url)
                response.raise_for_status()
                total_requests += 1
                
                data = response.json()
                if 'data' not in data or 'items' not in data['data']:
                    print("\nNo more data available from API")
                    break
                    
                new_tokens = data['data']['items']
                if not new_tokens:
                    print("\nNo more tokens available")
                    break
                
                tokens_in_window = 0
                oldest_time = last_token_time
                
                # Process each token
                for token in new_tokens:
                    token_time = int(datetime.fromisoformat(
                        token['liquidityAddedAt'].replace('Z', '+00:00')
                    ).timestamp())
                    
                    oldest_time = min(oldest_time, token_time)
                    
                    if start_timestamp <= token_time <= end_timestamp:
                        tokens_in_window += 1
                        all_tokens.append(token)
                
                last_token_time = oldest_time - 1  # Subtract 1 second to ensure no duplicates
                
                if tokens_in_window == 0:
                    print("\nNo tokens found in the specified time window")
                    break
                
            except requests.exceptions.HTTPError as e:
                print(f"\nHTTP Error: {e}")
                break
            except Exception as e:
                print(f"\nError: {e}")
                break
        
        # Convert to DataFrame
        df_full = pd.DataFrame(all_tokens)
        
        if not df_full.empty:
            # Convert timestamp to datetime
            df_full['liquidityAddedAt'] = pd.to_datetime(df_full['liquidityAddedAt'])
            
            print("\nTime range of fetched tokens:")
            print(f"Earliest: {df_full['liquidityAddedAt'].min()}")
            print(f"Latest: {df_full['liquidityAddedAt'].max()}")
            
            # Apply filters
            mask = (
                df_full['name'].notna() & 
                (df_full['name'] != 'None') & 
                df_full['logoURI'].notna() & 
                (df_full['logoURI'] != 'None') & 
                (df_full['liquidity'].astype(float) >= liquidity_filter)
            )
            df_filtered = df_full[mask]
            
            return df_full, df_filtered
        else:
            return pd.DataFrame(), pd.DataFrame()

# Example usage:
target_unix_time = 1733388916  # Provided by Birdeye
time_window_minutes = 10  # Search within 10 minutes before and after
liquidity_filter = 0

print("Fetching token data around the specified time...")
df_full, df_filtered = get_new_listings_around_time(target_unix_time, time_window_minutes, API_Key, liquidity_filter)

# Check for the specific token
if not df_full.empty and 'address' in df_full.columns:
    stress_guy_matches = df_full[df_full['address'] == 'dHadSsY2dvsoEDwDRP7rgq7EVYRZZ17qvpwkwsQpump']
    print("\nSTRESS GUY Token Information (Exact address match):")
    if not stress_guy_matches.empty:
        print(stress_guy_matches[['address', 'name', 'symbol', 'liquidity', 'liquidityAddedAt']].to_string())
    else:
        print("Token not found with exact address match")
else:
    print("\nNo tokens found in the specified time window")

Fetching token data around the specified time...

Fetching tokens around 2024-12-05 08:55:16+00:00 with a window of 10 minutes
Start timestamp: 1733388316
End timestamp: 1733389516

No tokens found in the specified time window

Time range of fetched tokens:
Earliest: 2024-12-05 08:45:20
Latest: 2024-12-05 09:05:10

STRESS GUY Token Information (Exact address match):
Token not found with exact address match


Top Traders

In [10]:
def get_top_traders(address, time_frame, sort_type, sort_by, total_traders, API_Key):
    """
    Fetch top traders data for a specific token.

    Args:
    address (str): The token address to fetch top traders for.
    time_frame (str): Time frame for the data (30m, 1h, 4h, 6h, 8h, 12h, 24h).
    sort_type (str): Sort order (asc, desc).
    sort_by (str): Sort criterion (trade, volume).
    total_traders (int): Number of top traders to retrieve.
    API_Key (str): Your Birdeye API key.

    Returns:
    pd.DataFrame: A DataFrame containing the top traders data.
    """
    limit = 10
    all_traders = []

    for offset in range(0, total_traders, limit):
        url = f"https://public-api.birdeye.so/defi/v2/tokens/top_traders?address={address}&time_frame={time_frame}&sort_type={sort_type}&sort_by={sort_by}&offset={offset}&limit={limit}"

        headers = {
            "accept": "application/json",
            "X-API-KEY": API_Key
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = json.loads(response.text)
            if 'data' in data and 'items' in data['data']:
                all_traders.extend(data['data']['items'])
            else:
                print(f"Unexpected response structure at offset {offset}")
                break
        else:
            print(f"Request failed with status code: {response.status_code} at offset {offset}")
            break

        if len(all_traders) >= total_traders:
            break

        # Respect rate limits
        time.sleep(1)

    return pd.DataFrame(all_traders[:total_traders])

# Example usage:
time_frame = "30m"  # 30m, 1h, 4h, 6h, 8h, 12h, 24h
sort_type = "desc"  # asc, desc
sort_by = "volume"  # trade, volume 
            # Note: trade is the number of transactions made, volume is the total volume of transactions made
total_traders = 20  # Change this to the number of top traders you want to retrieve

top_traders_df = get_top_traders(address, time_frame, sort_type, sort_by, total_traders, API_Key)
print(top_traders_df)

                                    tokenAddress  \
0   EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
1   EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
2   EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
3   EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
4   EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
5   EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
6   EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
7   EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
8   EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
9   EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
10  EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
11  EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
12  EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
13  EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
14  EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
15  EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
16  EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
17  EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   
18  EKpQGSJt

Top Markets for a Token

In [11]:
def get_markets(address, time_frame, sort_type, sort_by, total_markets, API_Key):
    """
    Fetch markets data for a specific token.

    Args:
    address (str): The token address to fetch markets for.
    time_frame (str): Time frame for the data (30m, 1h, 4h, 6h, 8h, 12h, 24h).
    sort_type (str): Sort order (asc, desc).
    sort_by (str): Sort criterion (volume24h, liquidity).
    total_markets (int): Number of markets to retrieve.
    API_Key (str): Your Birdeye API key.

    Returns:
    pd.DataFrame: A DataFrame containing the markets data.
    """
    limit = 10
    all_markets = []

    for offset in range(0, total_markets, limit):
        url = f"https://public-api.birdeye.so/defi/v2/markets?address={address}&time_frame={time_frame}&sort_type={sort_type}&sort_by={sort_by}&offset={offset}&limit={limit}"

        headers = {
            "accept": "application/json",
            "X-API-KEY": API_Key
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = json.loads(response.text)
            if 'data' in data and 'items' in data['data']:
                all_markets.extend(data['data']['items'])
            else:
                print(f"Unexpected response structure at offset {offset}")
                break
        else:
            print(f"Request failed with status code: {response.status_code} at offset {offset}")
            break

        if len(all_markets) >= total_markets:
            break

        # Respect rate limits
        time.sleep(1)

    if all_markets:
        df = pd.DataFrame(all_markets[:total_markets])

        # Flatten nested dictionaries
        df['base_address'] = df['base'].apply(lambda x: x.get('address'))
        df['base_symbol'] = df['base'].apply(lambda x: x.get('symbol'))
        df['quote_address'] = df['quote'].apply(lambda x: x.get('address'))
        df['quote_symbol'] = df['quote'].apply(lambda x: x.get('symbol'))

        # Select and rename columns based on available data
        columns_to_display = {
            'address': 'Market Address',
            'name': 'Name',
            'base_symbol': 'Base Symbol',
            'quote_symbol': 'Quote Symbol',
            'liquidity': 'Liquidity',
            'volume24h': 'Volume 24h',
            'price': 'Price',
            'trade24h': 'Trades 24h',
            'uniqueWallet24h': 'Unique Wallets 24h',
            'source': 'Source'
        }
        
        # Only include columns that exist in the DataFrame
        available_columns = [col for col in columns_to_display.keys() if col in df.columns]
        df = df[available_columns].rename(columns={col: columns_to_display[col] for col in available_columns})

        return df
    else:
        return pd.DataFrame()

# Example usage:
time_frame = "24h"  # 30m, 1h, 4h, 6h, 8h, 12h, 24h 
sort_type = "desc"  # asc, desc
sort_by = "volume24h"  # volume24h, liquidity
total_markets = 50  # Change this to the number of markets you want to retrieve

markets_df = get_markets(address, time_frame, sort_type, sort_by, total_markets, API_Key)
print(markets_df)


                                  Market Address          Name Base Symbol  \
0   D6NdKrKNQPmRZCCnG1GqXtF7MMoHB7qR6GU5TkG59Qz1      SOL-$WIF         SOL   
1   G4AGsrzHQWrPRELJdyr8CrWbA3PjvtGDVqMQhZcoaarr      $WIF-SOL        $WIF   
2   CwGmEwYFo7u5D7vghGwtcCbRToWosytaZa3Ys3JAto6J     $WIF-USDC        $WIF   
3   9xERPkyJuPBnffKX2SswG6r25sJMSyD4hTDqgm8d5QoV     $WIF-USDC        $WIF   
4   BKLhZ5NrFhCjViC4wyAMXBNsJFHbFfYujo3TtUmBxTH3      $WIF-SOL        $WIF   
5   EP2ib6dYdEeqD8MfE2ezHCxX3kP3K2eLKkirfPm5eyMx      $WIF-SOL        $WIF   
6   5KA7eEtZKSyGHyFwrNQZfFASDXtyPCtYCu5jwMBsQcSQ      $WIF-SOL        $WIF   
7   6ojSigXF7nDPyhFRgmn3V9ywhYseKF9J32ZrranMGVSX     $WIF-USDC        $WIF   
8   BuavWdfsNTfmEQbnPt2PLc51B7pifRNhqNiDUtGLeNNn      SOL-$WIF         SOL   
9   AishPhdJXUHpo3zg2gt5AVfdUg3MPmf8DV8FButvbRJ6   POPCAT-$WIF      POPCAT   
10  CN8M75cH57DuZNzW5wSUpTXtMrSfXBFScJoQxVCgAXes     $WIF-USDC        $WIF   
11  CyJG5PGm5hKrdnmszBUJQfjpSHVJqwUSYffXyTyTAyXv   POPCAT-$WIF  

### Wallet APIs

Supported Chains

In [12]:

url = "https://public-api.birdeye.so/v1/wallet/list_supported_chain"

headers = {
    "accept": "application/json",
    "X-API-KEY": API_Key
}

response = requests.get(url, headers=headers)

print(response.text)

{"success":true,"data":["solana","ethereum","arbitrum","avalanche","bsc","optimism","polygon","base","zksync"]}


Wallet Portfolio Multichain

In [13]:
def get_multichain_wallet_tokens(wallet, multichain, API_Key):
    """
    Fetch multichain wallet token list.

    Args:
    wallet (str): The wallet address to fetch tokens for.
    multichain (str): Comma-separated list of chains to query.
    API_Key (str): Your Birdeye API key.

    Returns:
    pd.DataFrame: A DataFrame containing the multichain wallet token list.
    """
    url = f"https://public-api.birdeye.so/v1/wallet/multichain_token_list?wallet={wallet}"

    headers = {
        "accept": "application/json",
        "x-chains": multichain,
        "X-API-KEY": API_Key
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = json.loads(response.text)
        if 'data' in data:
            all_tokens = []
            for chain, chain_data in data['data'].items():
                if isinstance(chain_data, list):
                    for token in chain_data:
                        if isinstance(token, dict):
                            token['chain'] = chain
                            all_tokens.append(token)
                elif isinstance(chain_data, dict):
                    chain_data['chain'] = chain
                    all_tokens.append(chain_data)
            
            if all_tokens:
                df = pd.DataFrame(all_tokens)
                
                # Reorder columns to have 'chain' first
                columns = ['chain'] + [col for col in df.columns if col != 'chain']
                df = df[columns]
                
                return df
            else:
                print("No tokens found in the wallet.")
                return pd.DataFrame()
        else:
            print("Unexpected response structure. 'data' key not found.")
            print(json.dumps(data, indent=2))
            return pd.DataFrame()
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.text)
        return pd.DataFrame()

# Example usage:
wallet = d.sol_wallet
multichain = "solana,ethereum,bsc" # "solana,ethereum,arbitrum,avalanche,bsc,optimism,polygon,base,zksync"
df = get_multichain_wallet_tokens(wallet, multichain, API_Key)
print(df)


   chain                                       address  decimals   balance  \
0  items   So11111111111111111111111111111111111111111         9  34723267   
1  items  EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm         6   1260929   

   uiAmount chainId       name symbol  \
0  0.034723  solana        SOL    SOL   
1  1.260929  solana  dogwifhat   $WIF   

                                             logoURI    priceUsd  valueUsd  \
0  https://wsrv.nl/?w=128&h=128&default=1&url=htt...  156.297362  5.427155   
1  https://wsrv.nl/?w=128&h=128&default=1&url=htt...    2.608506  3.289141   

                                                icon  
0                                                NaN  
1  https://wsrv.nl/?w=128&h=128&default=1&url=htt...  


Wallet Transaction History (Multichain)

In [14]:
def get_multichain_wallet_transactions(wallet, multichain, API_Key):
    """
    Fetch multichain wallet transaction list.

    Args:
    wallet (str): The wallet address to fetch transactions for.
    multichain (str): Comma-separated list of chains to query.
    API_Key (str): Your Birdeye API key.

    Returns:
    pd.DataFrame: A DataFrame containing the multichain wallet transaction list.
    """
    url = f"https://public-api.birdeye.so/v1/wallet/multichain_tx_list?wallet={wallet}"

    headers = {
        "accept": "application/json",
        "x-chains": multichain,
        "X-API-KEY": API_Key
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = json.loads(response.text)
        if 'data' in data:
            all_transactions = []
            for chain, chain_data in data['data'].items():
                for tx in chain_data:
                    tx['chain'] = chain
                    all_transactions.append(tx)
            
            if all_transactions:
                df = pd.DataFrame(all_transactions)
                
                # Convert blockTime to datetime
                df['datetime'] = pd.to_datetime(df['blockTime'])
                
                # Reorder columns to have 'chain' and 'datetime' first
                columns = ['chain', 'datetime'] + [col for col in df.columns if col not in ['chain', 'datetime', 'blockTime']]
                df = df[columns]
                
                return df
            else:
                print("No transactions found for the wallet.")
                return pd.DataFrame()
        else:
            print("Unexpected response structure. 'data' key not found.")
            print(json.dumps(data, indent=2))
            return pd.DataFrame()
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.text)
        return pd.DataFrame()

# Example usage:
wallet = d.sol_wallet
multichain = "solana,ethereum,bsc" # "solana,ethereum,arbitrum,avalanche,bsc,optimism,polygon,base,zksync"
df = get_multichain_wallet_transactions(wallet, multichain, API_Key)
print(df)

     chain                  datetime  \
0   solana 2024-10-18 14:35:27+00:00   
1   solana 2024-10-18 01:07:56+00:00   
2   solana 2024-10-17 21:01:00+00:00   
3   solana 2024-10-17 21:01:00+00:00   
4   solana 2024-10-17 21:00:54+00:00   
5   solana 2024-10-17 03:09:33+00:00   
6   solana 2024-10-16 03:09:12+00:00   
7   solana 2024-10-16 03:08:25+00:00   
8   solana 2024-10-16 03:07:58+00:00   
9   solana 2024-10-16 03:07:29+00:00   
10  solana 2024-10-16 03:05:30+00:00   
11  solana 2024-10-16 03:05:12+00:00   
12  solana 2024-10-16 02:54:19+00:00   
13  solana 2024-10-16 02:48:50+00:00   
14  solana 2024-10-16 01:58:12+00:00   
15  solana 2024-10-16 01:58:00+00:00   
16  solana 2024-10-16 01:08:52+00:00   
17  solana 2024-10-16 01:03:12+00:00   
18  solana 2024-10-16 00:13:29+00:00   
19  solana 2024-10-15 23:54:43+00:00   
20  solana 2024-10-15 13:07:12+00:00   
21  solana 2024-10-15 03:16:45+00:00   
22  solana 2024-10-15 02:24:55+00:00   
23  solana 2024-10-15 00:25:27+00:00   
